In [28]:
import requests
from fake_useragent import UserAgent
import time
import json
import re
import sqlite3
from bs4 import BeautifulSoup
import datetime
import hashlib
import pandas as pd
#regex for extracting decimals from https://stackoverflow.com/questions/12117024/decimal-number-regular-expression-where-digit-after-decimal-is-optional

In [17]:
db_name='d2.db'

In [2]:
courses_table="""CREATE TABLE if not exists "courses" (
	"url"	TEXT UNIQUE,
	"title"	TEXT,
	"instructor"	TEXT,
	"average_rating"	REAL,
	"total_pages"	INTEGER,
	"scraped_pages"	INTEGER,
	PRIMARY KEY("url")
);"""

reviews_table="""
CREATE TABLE if not exists "reviews" (
	"id"	TEXT UNIQUE,
	"course_url"	INTEGER,
	"reviewer"	TEXT,
	"review_date"	TEXT,
	"review_unix"	REAL,
	"helpful_count"	INTEGER,
	"rating"	REAL,
	"review"	INTEGER,
	PRIMARY KEY("id")
);"""

In [10]:
def initialize_db():
    #this function creates a new database and creates the two empty tables
    conn=sqlite3.connect(db_name)
    cur=conn.cursor()
    cur.execute(courses_table)
    cur.execute(reviews_table)
    conn.commit()
    cur.close()
    conn.close()

def scrape_course_main_page(course_url,cur,conn,debug=False):
    #this function checks the main page of a course and fetch the meta data
    html=requests.get(course_url).content
    soup=BeautifulSoup(html)
    instructor=soup.find('div',attrs={'class':'rc-BannerInstructorInfo'}).find('span').text.strip()
    average_rating=soup.find('span',attrs={'class':'number-rating'}).text.strip()
    average_rating=re.findall(r'\d+\.?\d*',average_rating)[0]
    title=soup.find('div',attrs={'class':'BannerTitle'}).find('h1').text.strip()
    
    review_html=requests.get(course_url+'/reviews').content
    review_soup=BeautifulSoup(review_html)
    #this code below finds the navigation bar, and the last element is the total number of pages of reviews
    pg_nav_bar=review_soup.find('nav',attrs={'aria-label':'Pagination Controls'}).findAll('span')
    total_pages=int(pg_nav_bar[-2].text)
    
    if debug:
        print("Course title: ",title)
        print("Instructor: ",instructor)
        print("Average course rating: ",average_rating)
        print("Total pages: ",total_pages)
    
    cur.execute("update courses set title=?, instructor=?, average_rating=?, total_pages=? where url=?", (title,instructor,average_rating,total_pages,course_url))
    conn.commit()
    
def scrape_course_review(course_url,pgn,cur,conn,debug=False):
    #this funtion will scrape exactly one page of course review, denoted by the page number
    if pgn==400:
        return#page 400 and after are not accessible
    review_url=course_url+'/reviews?page={}'.format(pgn)
    html=requests.get(review_url).content
    soup=BeautifulSoup(html)
    for review in soup.findAll('div',attrs={'class':'review-page-review'}):
        #for each review ...
        helpful_count=review.find('button',attrs={'class':'review-helpful-button'}).text
        helpful_count=re.findall(r'\d+',helpful_count)
        #sometimes no one will upvote a review as "helpful", thus regex won't find any number
        if helpful_count!=[]:
            helpful_count=helpful_count[0]
        else:
            helpful_count=0
        review_text=review.find('div',attrs={'class':'reviewText'}).text
        reviewer=review.find('p',attrs={'class':'reviewerName'}).text.replace("By","").strip()
        review_date=review.find('div',attrs={'class':'dateOfReview'}).text
        review_timestamp=datetime.datetime.strptime(review_date,'%b %d, %Y').timestamp()
        star_box=review.find('div',attrs={'role':'img'})
        star_count=len(star_box.findAll('title',string='Filled Star'))
        if debug:
            print("Reviwer: ",reviewer)
            print("Review Date: ",review_date)
            print("Upvote: ",helpful_count)
            print("Rating: ",star_count)
            print("Review: ",review_text)
        #use md5 hashing to create an unique id for each review
        uid=review_url+reviewer+review_date+review_text
        uid=hashlib.md5(uid.encode('utf-8')).hexdigest()
        cur.execute("INSERT INTO reviews (id,course_url,reviewer,review_date,review_unix,helpful_count,rating,review) values (?,?,?,?,?,?,?,?) on CONFLICT(id) DO UPDATE SET  id=id",(uid,course_url,reviewer,review_date,review_timestamp,helpful_count,star_count,review_text))
        conn.commit()
    cur.execute("update courses set scraped_pages=? where url=?",(pgn,course_url))
    conn.commit()
    
def scrape_course_list(pgn,cur,conn,debug=False):
    #this function scrapes exactly one page of courses, denoted by pgn
    list_url='https://www.coursera.org/search?query=free&page={}&index=prod_all_products_term_optimization&allLanguages=English'.format(pgn)
    html=requests.get(list_url).content
    soup=BeautifulSoup(html)
    #the course list is actually returned by the webserver as a json object
    list_obj=json.loads(soup.find('script',attrs={'type':'application/ld+json'}).string)
    for course in list_obj['itemListElement']:
        cur.execute("INSERT INTO courses (url) Values (?) on CONFLICT(url) DO UPDATE SET  url=url",(course['url'],))
        conn.commit()
        if debug:
            print(course['url'])

In [11]:
def scrape_course_meta_data(to_scrape_course_page=True,to_scrape_course_list=True,pages_of_courses=10):
    #this function calls some of the above functions to scrape the meta data
    #calls initialize db to initialize data base
    initialize_db()
    conn=sqlite3.connect(db_name)
    cur=conn.cursor()
    #if we choose to scrape the course list from coursera
    if to_scrape_course_list:
        #then we plan to scrape 10 pages of courses
        for i in range(1,pages_of_courses+1):
            #for each page number, call scrape course list function and provide the page number to scrape
            #the function will store the course url in the database
            scrape_course_list(i,cur,conn)
            time.sleep(5)
    
    #if we choose to scrape the meta data for courses
    if to_scrape_course_page:
        #then for all of the course urls that we collected above
        for i in cur.execute("select url from courses where title is null").fetchall():
            course_url=i[0]
            #call the function to scrape meta data for that course, such as the instructor name etc..
            scrape_course_main_page(course_url,cur,conn)
            time.sleep(5)
        
    cur.close()
    conn.close()

In [15]:
def scrape_reviews():
    conn=sqlite3.connect(db_name)
    cur=conn.cursor()
    #after we scraped the meta data for each course, it's time to scrape the reviews
    #for each course that we collected meta data above
    for i in cur.execute("select url,total_pages,scraped_pages from courses where total_pages is not null").fetchall():
        course_url,total_pages,scraped_pages=i
        #scraped pages is initialized to None
        #after each page of review is scraped, this column is updated
        #this scraped pages is a "progress" counter
        #if the number of scraped pages equals to total pages of reviews, then we are done for this particular course
        if total_pages==scraped_pages:
            continue
        if scraped_pages==None:
            scraped_pages=0
        #iterate through the page numbers for pages that are yet to be scraped
        for pgn in range(scraped_pages+1,total_pages+1):
            if pgn>10:#only scrape 10 pages for each course to have some diversity
                continue
            #call scrape review function to scrape reviews in that page
            scrape_course_review(course_url,pgn,cur,conn)
            time.sleep(5)

In [41]:
def clean_names_and_export():
    conn=sqlite3.connect(db_name)
    cur=conn.cursor()
    export_data=[]
    #for each piece of review collected, only extract id, review text, and instructor name
    for i in cur.execute("select id,review,instructor from reviews left join courses on courses.url = reviews.course_url").fetchall():
        uid,review,instructor=i
        #some instructor name list is too long, thus coursera adds '+n  more instructors' at the end
        #those irrelevant strings must be removed
        instructor=instructor.replace('more instructor','').replace('more instructors','')
        #the if statement is removing '+n' in the instructor name string
        instructor_names=[i for i in instructor.split() if i[0]!='+']
        #some reviews only have number and need to be thrown out
        if type(review) == type(5):
            continue
        #strip the excessive spaces
        review=review.strip()
        for name_candidate in instructor_names:
            review=review.replace(name_candidate,'')
        export_data.append({'id':uid,'review':review})
    export_data=pd.DataFrame(export_data)
    export_data.to_csv('exported_data.tsv',sep='\t', index=False)

In [13]:
scrape_course_meta_data()

In [16]:
scrape_reviews()

In [42]:
clean_names_and_export()